In [1]:
import torch
from torchvision import transforms
from transformers import AutoModelForImageClassification, AutoFeatureExtractor, AutoFeatureExtractor, AutoTokenizer, AutoConfig
from PIL import Image
from io import BytesIO

config_path = 'config.json'
model_checkpoint = 'model/pytorch_model.bin'

config = AutoConfig.from_pretrained(config_path)
model = AutoModelForImageClassification.from_pretrained(model_checkpoint, config=config)
model.load_state_dict(torch.load(model_checkpoint))
model.eval()

def preprocess_image(image_path):
    transform = transforms.Compose([transforms.Resize((224,224)),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229,0.224, 0.255])])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image

image_path = 'test_data/cat001.jpg'
image = preprocess_image(image_path)
with torch.no_grad():
    output = model(image)

probability = torch.sigmoid(output.logits[0][7]).item()

print(probability)

0.3657042980194092


Здесь были попытки разобраться, надо решить

In [26]:
import os
import torch
from torchvision import transforms
from transformers import AutoModelForImageClassification, AutoConfig
from PIL import Image


# Укажите путь к проверочному датасету
test_data_path = 'test_data'

# Пути к модели и конфигурации
config_path = 'config.json'
model_checkpoint = 'model/pytorch_model.bin'

# Загрузка модели и конфигурации
config = AutoConfig.from_pretrained(config_path)
model = AutoModelForImageClassification.from_pretrained(model_checkpoint, config=config)
model.eval()

# Функция для предобработки изображения
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.255])
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image

# Функция для классификации изображения и вычисления вероятности
def classify_image(image_path, model):
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
    probability = torch.sigmoid(output.logits[0][7]).item()
    return probability

test_data_directory = 'test_data'
probabilities = []


for filename in os.listdir(test_data_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(test_data_directory, filename)
        probability = classify_image(image_path, model)
        probabilities.append(probability)

# Вывод вероятностей для каждого изображения
for i, probability in enumerate(probabilities):
    print(f"Image {i + 1}: Probability = {probability}")


Image 1: Probability = 0.4530305564403534


In [3]:
import os
import torch
from torchvision import transforms
from transformers import AutoModelForImageClassification, AutoConfig
from PIL import Image
import zipfile

# Укажите путь к проверочному датасету
test_data_path = 'test_data-2'

# Пути к модели и конфигурации
config_path = 'config.json'
model_checkpoint = 'model/pytorch_model.bin'

# Загрузка модели и конфигурации
config = AutoConfig.from_pretrained(config_path)
model = AutoModelForImageClassification.from_pretrained(model_checkpoint, config=config)
model.eval()


def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.255])
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image

def classify_image(image_path, model):
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
     # тут надо по классам разбить, но их много, а тут всего 1. в общем все непонятно как я написала
    probability = torch.sigmoid(output.logits[0][1]).item()
    return probability


test_data_directory = 'test_data'


actual_labels = [] 
predicted_probabilities = []

for filename in os.listdir(test_data_directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(test_data_directory, filename)
        
        if '_' in filename and '.' in filename:
            class_label = int(filename.split('_')[1].split('.')[0])
        else:
            class_label = 1 if filename.startswith('class_1') else 0
        
        probability = classify_image(image_path, model)
        
        
        actual_labels.append(class_label)
        predicted_probabilities.append(probability)


correct_predictions = 0
total_images = len(actual_labels)

for actual, predicted in zip(actual_labels, predicted_probabilities):
    if (actual == 1 and predicted >= 0.5) or (actual == 0 and predicted < 0.5):
        correct_predictions += 1

accuracy = correct_predictions / total_images
print("Accuracy:", accuracy)


Accuracy: 0.8198198198198198
